In [1]:
import pygame as p
%run ChessEngine.ipynb
%run ChessAI.ipynb

WIDTH = HEIGHT = 512
CELLS = 8
SQ_SIZE = HEIGHT // CELLS
MAX_FPS = 15
IMAGES = {}
GREEN = (76, 153, 0)
WHITE = (229, 255, 204)

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def loadImages():
    pieces = ['bR','bN','bB','bQ','bK','bp','wR','wN','wB','wQ','wK','wp']
    for piece in pieces:
        IMAGES[piece] = p.image.load("images/" + piece + ".png")

In [3]:
def main():
    p.init()
    screen = p.display.set_mode((WIDTH, HEIGHT))
    clock = p.time.Clock()
    running = True
    gs = GameState()
    # print(gs.board)
    validMoves = gs.getValidMoves()
    moveMade = False #to check if a move is made or not and only generate valid moves when a move is made
    loadImages()

    sqSelected = ()
    playerClicks = []
    gameOver = False
    playerOne = True #if human playing white, then true; if AI white then false
    playerTwo = False #if human playing black, then true; if AI black then false
    
    while running:
        isHumanTurn = (gs.whiteToMove and playerOne) or (not gs.whiteToMove and playerTwo)
        for event in p.event.get():
            if event.type == p.QUIT:
                running = False
            #mouse handlersss
            elif event.type == p.MOUSEBUTTONDOWN:
                if not gameOver and isHumanTurn:
                    location = p.mouse.get_pos()
                    col = location[0] // SQ_SIZE
                    row = location[1] // SQ_SIZE
                    if sqSelected == (row, col):
                        sqSelected = ()
                        playerClicks = []
                    else:
                        sqSelected = (row, col)
                        playerClicks.append(sqSelected)
                    if len(playerClicks) == 2:
                        move = Move(playerClicks[0], playerClicks[1], gs.board)
                        if move in validMoves:
                            gs.makeMove(move)
                            print(move.getNotation())
                            moveMade = True
                            playerClicks = []
                            sqSelected = ()
                        else:
                            playerClicks = [sqSelected]
            #key handlers
            elif event.type == p.KEYDOWN:
                if event.key == p.K_z:   #undo move with key press Z
                    gs.undoMove()
                    moveMade = True
                    print("Undo last move")

        ##Moves for AI
        if not gameOver and not isHumanTurn:
            AIMove = findRandomMove(validMoves)
            
            gs.makeMove(AIMove)
            print(gs.getNotation(AIMove))
            moveMade = True
            
        if moveMade:
            validMoves = gs.getValidMoves()
            moveMade = False
            
        drawGameState(screen, gs, validMoves, sqSelected)

        if gs.checkMate:
            gameOver = True
            if gs.whiteToMove:
                drawText(screen, "Black wins by Check Mate")
            else:
                drawText(screen, "White wins by Check Mate")
        elif gs.staleMate:
            gameOver = True
            drawText(screen, "Draw by Stalemate")
            
        clock.tick(MAX_FPS)
        p.display.flip() 
    p.quit()

In [4]:
def highlightSquares(screen, gs, validMoves, sqSelected):
    if sqSelected != ():
        row, col = sqSelected
        if gs.board[row][col][0] == ('w' if gs.whiteToMove else 'b'):
            s = p.Surface((SQ_SIZE, SQ_SIZE))
            s.set_alpha(100) #transparency
            s.fill(p.Color("blue"))
            screen.blit(s, (col * SQ_SIZE, row * SQ_SIZE))
            s.fill(p.Color("yellow"))
            for move in validMoves:
                if move.startRow == row and move.startCol == col:
                    screen.blit(s, (move.endCol * SQ_SIZE, move.endRow * SQ_SIZE))

In [5]:
def drawGameState(screen, gs, validMoves, sqSelected):
    drawSquares(screen)
    highlightSquares(screen, gs, validMoves, sqSelected)
    drawPieces(screen, gs.board)
    

In [6]:
def drawSquares(screen):
    colors = [p.Color(WHITE),p.Color(GREEN)]
    for row in range(CELLS):
        for col in range(CELLS):
            color = colors[(row + col) % 2]
            p.draw.rect(screen, color, p.Rect(col * SQ_SIZE, row * SQ_SIZE, SQ_SIZE, SQ_SIZE))

In [7]:
def drawPieces(screen, board):
    for row in range(CELLS):
        for col in range(CELLS):
            piece = board[row][col]
            if piece != "--":
                screen.blit(IMAGES[piece], p.Rect(col * SQ_SIZE, row * SQ_SIZE, SQ_SIZE, SQ_SIZE))
                

In [8]:
def drawText(screen, text):
    font = p.font.SysFont("Helvitca", 32, True, False)
    textObject = font.render(text, 0, p.Color("black"))
    textLocation = p.Rect(0, 0, WIDTH, HEIGHT).move(WIDTH/2 - textObject.get_width()/2, HEIGHT/2 - textObject.get_height()/2)
    screen.blit(textObject, textLocation)

In [9]:
if __name__ == "__main__":
    main()

e2e4
d2d4
e4e5
